

```
https://blog.futuresmart.ai/langgraph-rag-agent-tutorial-basics-to-advanced-multi-agent-ai-chatbot?source=more_series_bottom_blogs#heading-step-3-building-a-basic-langgraph-chatbot
```



In [ ]:
!pip install langgraph langchain langchain_openai langchain_community duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.0 MB/s eta 0:00:00


In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    """State for our two‑node agent"""
    messages: Annotated[list[BaseMessage], add_messages]


In [ ]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions. Use this for any math calculations."""
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

search_tool = DuckDuckGoSearchRun()

In [ ]:
from langchain_openai import ChatOpenAI
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7, api_key=openai_api_key)
tools = [calculator, search_tool]
llm_with_tools = llm.bind_tools(tools)  # `tools` already contains `calculator` and `search_tool`

In [ ]:
# The chatbot node – decide answer vs. tool
from typing import Annotated, TypedDict # Import necessary types
from langgraph.graph.message import add_messages # Import add_messages
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage # Import BaseMessage

def chatbot_node(state: AgentState) -> AgentState:
    """Gatekeeper: answer directly or request a tool"""
    system_message = (
        "You are a helpful assistant.\n"
        "Use the `web_search` tool for real‑time facts and `calculator` for maths.\n"
        "Otherwise answer directly."
    )

    messages = [
        {"role": "system", "content": system_message},
        *state["messages"],
    ]

    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}  # LangGraph merges this into the running state

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)  # automatically dispatches and streams results back


In [ ]:
from typing import Literal

def should_continue(state: AgentState) -> Literal["tools", "end"]:
    last = state["messages"][-1]
    return "tools" if getattr(last, "tool_calls", None) else "end"


In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(AgentState)
workflow.add_node("chatbot", chatbot_node)
workflow.add_node("tools",   tool_node)

workflow.add_edge(START, "chatbot")
workflow.add_conditional_edges("chatbot", should_continue, {"tools": "tools", "end": END})
workflow.add_edge("tools", "chatbot")  # come back after tools run

app = workflow.compile(checkpointer=MemorySaver())

In [ ]:
# Manual Test
def chat_with_agent(msg: str, thread_id="demo"):
    cfg = {"configurable": {"thread_id": thread_id}}
    state = {"messages": [HumanMessage(content=msg)]}
    result = app.invoke(state, cfg)
    print(result["messages"][-1].content)

chat_with_agent("What's 15% of 240?")
chat_with_agent("Search for recent news about artificial intelligence")


15% of 240 is 36.
Recent news about artificial intelligence includes various updates such as NVIDIA helping Germany lead Europe's AI manufacturing race, and discussions on how artificial intelligence is creating the next normal. Additionally, there are ongoing developments and insights into AI across industries including robotics, manufacturing, and enterprise. Notably, video game performers have reached a tentative deal after striking over AI-related issues. For more detailed and specific news, I can provide further summaries or articles. Would you like me to do that?
